# Overfitting e regolarizzazione
L'overfitting è un problema tipico del machine learning che si manifesta quando un modello si lega troppo ai dati di addestramento e fallisce nel generalizzare su dati nuovi.

L'overffiting è caratterizzato da:
* **Alta variaza**: le previsioni per modelli addestrati con diverse parti del dataset saranno molto diverse tra loro.
* **Basso bias**: l'errore per le predizioni sul set di addestramento è mediamente molto basso.
<img src="res/overfitting.png" width="500px" /><br>
In questo notebook utilizzremo scikit-learn e il Boston Housing Dataset per studiare da vicino il problema dell'overfitting ed imparare a constrastarlo.

### Librerie necessarie

In [1]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

### Importiamo il dataset

In [2]:
boston = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data", sep='\s+', 
                     names=["CRIM", "ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PRATIO","B","LSTAT","MEDV"])
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
X = boston.drop('MEDV',axis=1).values
Y = boston['MEDV'].values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=0)

### Creiamo le features polinomiali
Per correggere l'overfitting prima dobbiamo causarlo, un buon modo è aumentare la complessità del nostro modello aumentando il numero di features utilizzando i polinomi.

In [14]:
polyfeats = PolynomialFeatures(degree=2)
X_train_poly = polyfeats.fit_transform(X_train)
X_test_poly = polyfeats.transform(X_test)

print("Numero di esempi nel test di addestramento: "+str(X_train_poly.shape[0]))
print("Numero di features: "+str(X_train_poly.shape[1]))

Numero di esempi nel test di addestramento: 354
Numero di features: 105


### Standardizziamo i dati
**NOTA BENE** Per applicare la regolarizzazione è sempre necessario portare i dati sulla stessa scala.

In [6]:
ss = StandardScaler()
X_train_poly = ss.fit_transform(X_train_poly)
X_test_poly = ss.transform(X_test_poly)

Adesso il nostro set di addestramento contiene 354 e 105 features, abbastanza complesso !

### Riconoscere l'overfitting
Evidenziare un problema di overfitting è molto semplice, un modello che ne soffre avrà memorizzato la struttura dei dati di addestramento, piuttosto che imparare da essi, quindi l'errore per le predizioni sul train set sarà molto basso, invece fallirà nel generalizzare, perciò l'errore nel test set sarà decisamente più alto.<br><br>
Quindi per riconoscere l'overfitting è sufficente confrontare questi due valori, scriviamo una funzione che ci permette di farlo in modo da non dover scrivere più volte lo stesso codice.

In [7]:
def overfit_eval(model, X, Y):
    
    """
    model: il nostro modello predittivo già addestrato
    X: una tupla contenente le prorietà del train set e test set (X_train, X_test)
    Y: una tupla contenente target del train set e test set (Y_train, Y_test)
    """
    
    Y_pred_train = model.predict(X[0])
    Y_pred_test = model.predict(X[1])
    
    mse_train = mean_squared_error(Y[0], Y_pred_train)
    mse_test = mean_squared_error(Y[1], Y_pred_test)

    r2_train = r2_score(Y[0], Y_pred_train)
    r2_test = r2_score(Y[1], Y_pred_test)    
    
    print("Train set:  MSE="+str(mse_train)+" R2="+str(r2_train))
    print("Test set:  MSE="+str(mse_test)+" R2="+str(r2_test))    

### Regressione lineare non regolarizzata
Cominciamo eseguendo una regressione lineare (in realtà si tratta di una regressione polinomiale) senza applicare la regolarizzazione.

In [8]:
ll = LinearRegression()
ll.fit(X_train_poly, Y_train)

overfit_eval(ll, (X_train_poly, X_test_poly),(Y_train, Y_test))

Train set:  MSE=4.116985074931631 R2=0.9514303225914079
Test set:  MSE=29.550444547162407 R2=0.6451057885504222


Il modello predice in maniera estremamente (o meglio dire eccessivamente) accurata i dati del train set, mentre è molto più scarso sul test set. Siamo di fronte ad un caso di overfitting.

## Regolarizzazione L2: Ridge Regression
La ridge regression è un modello di regressione lineare che applica la **regolarizzazione L2**, la quale consiste nell'aggiungere una penalità per i pesi nella funzione di costo durante la fase di addestramento.<br>
La penalità è data dalla somma dei quadrati dei pesi:
$$\lambda\sum_{j=1}^{M}W_j^2$$<br>
**Lambda** (conosciuto anche come **alpha**) è il **parametro di regolarizzazione** ed è un'altro iperparametro.
Eseguiamo diverse Ridge regression per diversi valori di alpha.

In [10]:
from sklearn.linear_model import Ridge

alphas = [0.0001, 0.001, 0.01, 0.1 ,1 ,10] #alpha corrispone a lambda

for alpha in alphas:
    print("Alpha="+str(alpha))
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_poly, Y_train)

    overfit_eval(ridge, (X_train_poly, X_test_poly),(Y_train, Y_test))

Alpha=0.0001
Train set:  MSE=4.099263404868843 R2=0.9516393920396632
Test set:  MSE=28.917618463358185 R2=0.6527058878869307
Alpha=0.001
Train set:  MSE=4.113502509945512 R2=0.9514714077677813
Test set:  MSE=28.420009267326737 R2=0.6586820627276777
Alpha=0.01
Train set:  MSE=4.208206127238454 R2=0.9503541522864941
Test set:  MSE=26.813295018132663 R2=0.6779783405072717
Alpha=0.1
Train set:  MSE=4.747028830953716 R2=0.9439974508597079
Test set:  MSE=23.63175511737236 R2=0.7161879211609108
Alpha=1
Train set:  MSE=5.875947305341872 R2=0.9306791596529944
Test set:  MSE=17.634584627530085 R2=0.7882125937009261
Alpha=10
Train set:  MSE=8.81275552173788 R2=0.896032488585423
Test set:  MSE=17.15971577477419 R2=0.7939156621191287


La Ridge regression, applicando la regolarizzazione L2, ci permette di ridurre l'overfitting e portare l'R2 fino ad un valore di 0.791 per alpha uguale a 10.

## Regolarizzazione L1: Lasso
Lasso è un modello di regressione lineare che applica la regolarizzazione L1, questa funziona in egual modo alla L2, con la differenza che il termine di regolarizza sarà dato dalla somma del valore assoluto dei pesi:
$$\lambda\sum_{j=1}^{M}|W_j|$$<br>
e viene sempre applicato alla funzione di costo durante la fase di addestramento

In [12]:
from sklearn.linear_model import Lasso

alphas = [0.0001, 0.001, 0.01, 0.1 ,1 ,10] #alpha corrisponde a lambda

for alpha in alphas:
    print("Alpha="+str(alpha))
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_poly, Y_train)

    overfit_eval(lasso, (X_train_poly, X_test_poly),(Y_train, Y_test))

Alpha=0.0001
Train set:  MSE=5.391123652697102 R2=0.9363988132296843
Test set:  MSE=29.701776720600787 R2=0.6432883230881463
Alpha=0.001
Train set:  MSE=5.407317548867123 R2=0.9362077675254491
Test set:  MSE=28.78801855730653 R2=0.6542623536919963
Alpha=0.01
Train set:  MSE=6.0638588169003205 R2=0.9284622943178907
Test set:  MSE=22.933242012656393 R2=0.7245769068863108
Alpha=0.1
Train set:  MSE=11.833211121207539 R2=0.860398996740507
Test set:  MSE=19.29615234281638 R2=0.7682575380960781
Alpha=1
Train set:  MSE=21.59098506709198 R2=0.7452827346818105
Test set:  MSE=27.25804314512913 R2=0.6726370152499754
Alpha=10
Train set:  MSE=84.76451346994796 R2=0.0
Test set:  MSE=83.76673764512785 R2=-0.0060197319476869016


/Users/giuseppe/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso ci permette di ottenere un modello ancora migliore, con un R2 di 0.803 per Lambda uguale a 0.1.<br>
Da notare che per valori di lambda più grandi il modello peggiora, questo perché l'effetto della regolarizzazione sarà molto pesante e buona parte dei pesi saranno portati a 0.

## L2 ed L1 insieme: ElasticNet
ElasticNet è un modello di regressione lineare che implementa entrambe le tecniche di regolarizzazone L2 ed L1.<br>
Tramite il parametro <span style="font-family: Monaco">l1_ration</span> possiamo controllare l'effetto delle due regolarizzazione
 * **<span style="font-family: Monaco">l1_ration>0.5</span>** l'effetto della regolarizzazione L1 sarà più intenso rispetto alla L2.
 * **<span style="font-family: Monaco">l1_ration<0.5</span>** l'effetto della regolarizzazione L2 sarà più intenso rispetto alla L1.

In [13]:
from sklearn.linear_model import ElasticNet

alphas = [0.0001, 0.001, 0.01, 0.1 ,1 ,10]

for alpha in alphas:
    print("Lambda is: "+str(alpha))
    elastic = ElasticNet(alpha=alpha, l1_ratio=0.5)
    elastic.fit(X_train_poly, Y_train)
    overfit_eval(elastic, (X_train_poly, X_test_poly),(Y_train, Y_test))

Lambda is: 0.0001
Train set:  MSE=5.391059281137906 R2=0.9363995726460551
Test set:  MSE=29.466017582883637 R2=0.6461197374561646
Lambda is: 0.001
Train set:  MSE=5.463124643400419 R2=0.9355493894819877
Test set:  MSE=26.238997938658404 R2=0.6848755196286389
Lambda is: 0.01
Train set:  MSE=6.669947875220288 R2=0.921312025490655
Test set:  MSE=15.78442472698635 R2=0.8104325991533365
Lambda is: 0.1
Train set:  MSE=12.092531251957979 R2=0.8573396960952863
Test set:  MSE=20.123693597792272 R2=0.7583189532244381
Lambda is: 1
Train set:  MSE=21.178857007859765 R2=0.7501447700119411
Test set:  MSE=27.923580301576486 R2=0.6646440632674491
Lambda is: 10
Train set:  MSE=70.28359861834348 R2=0.1708369960353573
Test set:  MSE=69.68198552608109 R2=0.16313498207951238


/Users/giuseppe/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Utilizzando ElasticNet, e quindi entrambe le regolarizzazioni, abbiamo ottenuto un modello ancora migliore, con un R2 di 0.81 sul test set e 0.92 sul test set.<br>
Abbiamo il nostro vincitore!

## Che differenza c'è tra la regolarizzazione L2 ed L1 ?

La differenza principale tra le due tecniche di regolarizzazione viste è la seguente:
* La regolarizzazione L2 riduce la magnitudine dei pesi a valori più bassi.
* La regolarizzazione L1 elimina le feature più deboli portando il loro peso a 0.
Nella pratica la L2 porta quasi sempre a migliori risultati, ma utilizzarle entrambe con ElasticNet è anche un ottima soluzione.